In [1]:
!pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-ya3wej7h
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-ya3wej7h
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.4 MB/s eta 0:00:00
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456765 sha256=a4e03a0f57a5b7a85964cc482058174b3889c7257f72c9982f4277805899662e
  Stored in directory: /tmp/pip-ephem-wheel-cache-l_gnnk5g/wheels/64/f4/2c/509a6c31b8ebde891a81029fd94f199b1b92f0e7cfc20d417a
Successfully built BLEURT


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from bleurt import score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score

## 1-load dataset

In [4]:
e_f_transformare = pd.read_csv('e-f-dataset.csv')

In [5]:
e_f_transformare.shape

(900, 2)

In [6]:
print("First few rows of the dataset:")
print(e_f_transformare.head())

print("\nBasic statistics:")
print(e_f_transformare.describe())


First few rows of the dataset:
                                             Column1  \
0                                            Column1   
1                                            Column1   
2                                            Column1   
3                                                 en   
4  Changing Lives | Changing Society | How It Wor...   

                                             Column2  
0                                            Column2  
1                                            Column2  
2                                            Column2  
3                                                 fr  
4  Il a transformé notre vie | Il a transformé la...  

Basic statistics:
        Column1  Column2
count       900      900
unique      897      896
top     Column1  Column2
freq          3        3


## 2-Data Preprocessing  


#### 1-Tokenize the text

In [7]:
def tokenize_and_normalize(text):
    if isinstance(text, str):
        tokens = word_tokenize(text.lower())
        print(f"Original text: {text}, Tokens: {tokens}")
        return tokens

    else:
        return []

In [7]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
e_f_transformare['Column1_tokenized'] = e_f_transformare['Column1'].apply(tokenize_and_normalize)
e_f_transformare['Column2_tokenized'] = e_f_transformare['Column2'].apply(tokenize_and_normalize)


Original text: Column1, Tokens: ['column1']
Original text: Column1, Tokens: ['column1']
Original text: Column1, Tokens: ['column1']
Original text: en, Tokens: ['en']
Original text: Changing Lives | Changing Society | How It Works | Technology Drives Change Home | Concepts | Teachers | Search | Overview | Credits | HHCC Web | Reference | Feedback Virtual Museum of Canada Home Page, Tokens: ['changing', 'lives', '|', 'changing', 'society', '|', 'how', 'it', 'works', '|', 'technology', 'drives', 'change', 'home', '|', 'concepts', '|', 'teachers', '|', 'search', '|', 'overview', '|', 'credits', '|', 'hhcc', 'web', '|', 'reference', '|', 'feedback', 'virtual', 'museum', 'of', 'canada', 'home', 'page']
Original text: Site map, Tokens: ['site', 'map']
Original text: Feedback, Tokens: ['feedback']
Original text: Credits, Tokens: ['credits']
Original text: Français, Tokens: ['français']
Original text: What is light ?, Tokens: ['what', 'is', 'light', '?']
Original text: The white light spectrum 

Original text: Il est constitué de 13 antennes paraboliques montées sur une plateforme de six mètres de diamètre qui peut être pointée n'importe où dans le ciel., Tokens: ['il', 'est', 'constitué', 'de', '13', 'antennes', 'paraboliques', 'montées', 'sur', 'une', 'plateforme', 'de', 'six', 'mètres', 'de', 'diamètre', 'qui', 'peut', 'être', 'pointée', "n'importe", 'où', 'dans', 'le', 'ciel', '.']
Original text: Chaque antenne fait 90 centimètres de diamètre et loge dans un cylindre protecteur fermé d'un couvercle de Teflon laissant passer les micro-ondes., Tokens: ['chaque', 'antenne', 'fait', '90', 'centimètres', 'de', 'diamètre', 'et', 'loge', 'dans', 'un', 'cylindre', 'protecteur', 'fermé', "d'un", 'couvercle', 'de', 'teflon', 'laissant', 'passer', 'les', 'micro-ondes', '.']
Original text: En 2002, le télescope détecte une configuration dans la distribution du rayonnement de fond cosmologique révélant des structures beaucoup plus petites que toutes celles observées jusqu'à présent., T

#### 2-Remove stop words

In [1]:
# import nltk
# nltk.download('stopwords')


In [2]:
# stop_words = set(stopwords.words('english'))

# e_f_transformare['Column1'] = e_f_transformare['Column1_tokenized'].apply(lambda x: [word for word in x if word not in stop_words])
# e_f_transformare['Column2'] = e_f_transformare['Column2_tokenized'].apply(lambda x: [word for word in x if word not in stop_words])


In [11]:
print("\nData after Preprocessing:")
print(e_f_transformare[['Column1', 'Column2']].head())


Data after Preprocessing:
                                             Column1  \
0                                          [column1]   
1                                          [column1]   
2                                          [column1]   
3                                               [en]   
4  [changing, lives, |, changing, society, |, wor...   

                                             Column2  
0                                          [column2]  
1                                          [column2]  
2                                          [column2]  
3                                               [fr]  
4  [il, transformé, notre, vie, |, il, transformé...  


# --------------------------------------------------------------------------------

#### 3-BERT embeddings

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [19]:
def tokenize_and_embed(text):
    if isinstance(text, list) and len(text) > 0:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings.squeeze().detach().numpy()
    else:
        return np.zeros(768)


##### Apply tokenize_and_embed function to create new columns

In [23]:
e_f_transformare['Column1_embeddings'] = e_f_transformare['Column1'].apply(tokenize_and_embed)
e_f_transformare['Column2_embeddings'] = e_f_transformare['Column2'].apply(tokenize_and_embed)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
print("\nUpdated DataFrame with BERT embeddings:")
print(e_f_transformare[['Column1', 'Column1_embeddings', 'Column2', 'Column2_embeddings']].head())


Updated DataFrame with BERT embeddings:
                                             Column1  \
0                                          [column1]   
1                                          [column1]   
2                                          [column1]   
3                                               [en]   
4  [changing, lives, |, changing, society, |, wor...   

                                  Column1_embeddings  \
0                         [-0.22546472, 0.017723754]   
1                         [-0.22546472, 0.017723754]   
2                         [-0.22546472, 0.017723754]   
3                          [-0.21105137, 0.10432148]   
4  [[-0.19938275, 0.07157811], [-0.17543136, -0.0...   

                                             Column2  \
0                                          [column2]   
1                                          [column2]   
2                                          [column2]   
3                                               [fr]   
4  [i

In [25]:
print(e_f_transformare.columns)

Index(['Column1', 'Column2', 'Column1_tokenized', 'Column2_tokenized',
       'Column1_embeddings', 'Column2_embeddings'],
      dtype='object')


### 4-train model


#### Split the data into training and testing sets

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    e_f_transformare[['Column1_embeddings', 'Column2_embeddings']],
    e_f_transformare[['Column1', 'Column2']],
    test_size=0.2,
    random_state=42
)


In [27]:
flat_y_train = [item for sublist in y_train['Column1'] + y_train['Column2'] for item in sublist]
flat_y_test = [item for sublist in y_test['Column1'] + y_test['Column2'] for item in sublist]
all_labels = flat_y_train + flat_y_test
le = LabelEncoder()
le.fit(all_labels)
y_train_encoded = [le.transform(item) for item in y_train['Column1'] + y_train['Column2']]
y_test_encoded = [le.transform(item) for item in y_test['Column1'] + y_test['Column2']]
y_train_combined = np.concatenate(y_train_encoded)
y_test_combined = np.concatenate(y_test_encoded)


### Build a simple neural network model

In [28]:
model = Sequential()
model.add(Dense(256, input_dim=768*2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [29]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
print(X_train['Column1_embeddings'].shape)
print(X_train['Column2_embeddings'].shape)


(720,)
(720,)


In [32]:
X_train_combined = np.concatenate([
    np.array(X_train['Column1_embeddings'].tolist())[:, np.newaxis],
    np.array(X_train['Column2_embeddings'].tolist())[:, np.newaxis]
], axis=1)

X_test_combined = np.concatenate([
    np.array(X_test['Column1_embeddings'].tolist())[:, np.newaxis],
    np.array(X_test['Column2_embeddings'].tolist())[:, np.newaxis]
], axis=1)


<ipython-input-32-5983c38f3589>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(X_train['Column1_embeddings'].tolist())[:, np.newaxis],
<ipython-input-32-5983c38f3589>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(X_train['Column2_embeddings'].tolist())[:, np.newaxis]
<ipython-input-32-5983c38f3589>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype

#### train model

In [33]:
texts = []
for i in range(len(X_train_combined)):
    # Assuming X_train_combined contains pairs of embeddings
    text = f"{X_train_combined[i][0]} {X_train_combined[i][1]}"
    texts.append(text)


In [34]:

# Convert text data to BERT input format
input_ids = []
attention_masks = []
for text in texts:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128, 
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(y_train_combined[:len(input_ids)])  

print("Input IDs size:", input_ids.size())
print("Attention Masks size:", attention_masks.size())
print("Labels size:", labels.size())


Input IDs size: torch.Size([720, 128])
Attention Masks size: torch.Size([720, 128])
Labels size: torch.Size([720])


In [35]:
dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1, output_attentions=False, output_hidden_states=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=6e-6, eps=1e-8)
num_epochs = 10
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch

In [36]:
model.train()
for epoch in range(num_epochs):
    total_loss = 0

    for i, (ids, masks, labels) in enumerate(train_loader):
        ids = ids.to(device)
        masks = masks.to(device)
        labels = labels.to(device).float()

        outputs = model(ids, attention_mask=masks)
        logits = outputs.logits.squeeze(1)

        loss_fn = torch.nn.BCEWithLogitsLoss()
        loss = loss_fn(logits, labels)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        if (i+1) % 100 == 0:
            print('{} / {}'.format(len(train_loader)*epoch + (i+1), total_steps))

    print('Epoch: {}, Loss: {:.4f}'.format(epoch+1, total_loss / len(train_loader)))


Epoch: 1, Loss: -5934.2669
Epoch: 2, Loss: -12390.4960
Epoch: 3, Loss: -16493.1598
Epoch: 4, Loss: -19750.1727
Epoch: 5, Loss: -22150.2887
Epoch: 6, Loss: -23951.1718
Epoch: 7, Loss: -25298.1280
Epoch: 8, Loss: -26295.7875
Epoch: 9, Loss: -26878.4791
Epoch: 10, Loss: -27193.1517


#### Save the trained model

In [37]:
torch.save(model.state_dict(), 'BERT_classification_model.ckpt')

#### Evaluate the model using BLEU

In [62]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1, output_attentions=False, output_hidden_states=False)
model.load_state_dict(torch.load('BERT_classification_model.ckpt'))
model.to(device)
model.eval()

predictions = []
true_labels = []

with torch.no_grad():
    for ids, masks, labels in val_loader:
        ids = ids.to(device)
        masks = masks.to(device)

        outputs = model(ids, attention_mask=masks)
        logits = outputs.logits.squeeze(1)
        predictions.extend(logits.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions]

accuracy = accuracy_score(true_labels, binary_predictions)
print(f'Accuracy: {accuracy:.4f}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy: 0.0000
